In [28]:
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from statistics import mean
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import graphviz

In [2]:
dataset_path = 'dataset.csv'
dataset = pd.read_csv(dataset_path)

In [3]:
dataset

,Cografya,Yerlesim,AracYogunlugu,YayaYogunlugu,VeriHizi,Guvenilirlik,Gecikme,Kapsama,Standart
0,5,6,6,7,9,6,4,6,1
1,6,2,6,1,3,2,6,7,0
2,5,1,6,9,3,6,6,5,1
3,8,2,3,5,7,5,7,9,0
4,5,7,4,8,5,9,5,6,1
...,...,...,...,...,...,...,...,...,...
9995,4,6,4,4,1,9,9,9,0
9996,1,10,6,10,6,4,8,5,0
9997,4,4,2,6,4,4,9,5,0
9998,4,9,7,2,5,4,2,7,0


In [4]:
x = dataset.iloc[:, :-1]
y = dataset.iloc[:, -1:]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, shuffle=True, stratify=y)

In [8]:
num_folds = 5
skf = StratifiedKFold(n_splits=num_folds, random_state=42, shuffle=True)
criterion_list = ['gini', 'entropy']
results = []

for criterion in criterion_list:
    accuracy_list = []
    for train_cv_idx, val_idx in skf.split(x_train, y_train):
        x_train_cv, x_val = x_train.iloc[train_cv_idx], x_train.iloc[val_idx]
        y_train_cv, y_val = y_train.iloc[train_cv_idx].values.ravel(), y_train.iloc[val_idx].values.ravel()
        model = DecisionTreeClassifier(random_state=42, criterion=criterion)
        model.fit(x_train_cv, y_train_cv)
        accuracy_list.append(model.score(x_val, y_val))
    print('Validation accuracy for criterion =', criterion, '\t:', mean(accuracy_list))
    results.append(mean(accuracy_list))
    
max_val = max(results)
max_idx = results.index(max_val)

print('*' * 50)
print('The best validation accuracy was obtained with criterion =', criterion_list[max_idx], 'as', max_val)

Validation accuracy for criterion = gini 	: 0.846375
Validation accuracy for criterion = entropy 	: 0.847625
**************************************************
The best validation accuracy was obtained with criterion = entropy as 0.847625


In [9]:
model = DecisionTreeClassifier(random_state=42, criterion=criterion_list[max_idx])
model.fit(x_train, y_train.values.ravel())
pred = model.predict(x_test)

cm = confusion_matrix(y_test, pred)
print(cm)

[[1016  175]
 [ 160  649]]


In [10]:
print('Test accuracy:', accuracy_score(y_test, pred))

Test accuracy: 0.8325


In [11]:
label_names = ['3GPP', 'IEEE']
report = classification_report(y_test, pred, target_names=label_names)
print(report)

              precision    recall  f1-score   support

        3GPP       0.86      0.85      0.86      1191
        IEEE       0.79      0.80      0.79       809

    accuracy                           0.83      2000
   macro avg       0.83      0.83      0.83      2000
weighted avg       0.83      0.83      0.83      2000



In [35]:
dot_data = export_graphviz(model, out_file=None, feature_names=list(x.columns), class_names=label_names, 
                           filled=True, rounded=True, special_characters=True)

graph = graphviz.Source(dot_data)
graph.format = 'png'
graph.render('DT_visualized')

dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.619473 to fit


'DT_visualized.png'

In [34]:
graph